# Kütüphaneleri Yükleme

In [1]:
import pandas as pd #Veri manipülasyonu ve analizi için kullanılır.
import re #Düzenli ifadeleri işlemek için kullanılır.
import textstat #Metinlerin okunabilirliğini ve karmaşıklığını değerlendirmek için kullanılır.
from langdetect import detect #Metnin dilini tespit etmek için kullanılır.
import tkinter as tk #Kullanıcı ara yüzü oluşturmak için kullanılır.
from tkinter import messagebox #Basit iletişim kutuları oluşturur.
from tkinter import ttk #Listeleri arayüze yansıtmak için kullanılır.
from sklearn.feature_extraction.text import TfidfVectorizer #Metin verilerini sayısal özelliklere dönüştürmek için kullanılır.
from sklearn.metrics.pairwise import sigmoid_kernel #Veri benzerliğini hesaplamak için kullanılır.
from scipy import sparse #Seyrek matrislerle çalışmak için alt modül.

-> Gerekli kütüphaneler yüklenir. İşlevleri yarum satırlarında detaylandırılmıştır.

# Veri Setini Manipüle Etme

In [2]:
df = pd.read_csv("dataset.csv")

-> Kaggle platformunda bulunan "Song lyrics dataset" veri seti kullanılmıştır. Veri setine "https://www.kaggle.com/datasets/mehedihasan9021/movie-script-dataset/data" adresinden ulaşılabilir.

In [3]:
df.head()

,genre,lyrics,SongInfo
0,Christian,"Who am I, that the Lord of all the earth Woul...",CASTING CROWNS - WHO AM I LYRICS
1,Christian,Glory Revealed By His Wounds He was pierced ...,GLORY REVEALED - BY HIS WOUNDS LYRICS
2,Christian,Lord of heaven and earth Lord of all creation...,CAEDMON'S CALL - GOD OF WONDERS LYRICS
3,Christian,I can only imagine what it will be like When ...,MERCYME - I CAN ONLY IMAGINE LYRICS
4,Christian,I am not skilled to understand What God has w...,AARON SHUST - MY SAVIOR MY GOD LYRICS


-> Kullanılan veri seti işlenmemiş şarkı sözlerinden oluşmaktadır. Bu nedenle öncelikle şarkı sözleri kullanıabilir forma getirmiştir.

In [4]:
def metin_temizle(text):
    temiz_metin = re.sub(r"[^\w\s]", "", text) # Noktalama işaretlerini ve sayıları kaldırır.
    temiz_metin = temiz_metin.lower() # Büyük harfleri küçük harfe dönüştürür.
    return temiz_metin

-> Şarkı sözlerini temizlemek için oluşturulmuş bir fonksiyondur.

In [5]:
def dil_bul(text):
    try:
        dil_kodu = detect(text) # Cümledeki dilin kısa adını döndürür.
        return dil_kodu
    except:
        return None # Dil belirlenemezse NaN döndürür.

-> Veri setinde, kullanılacak kütüphanelerle uyumlu olmayan dillere ait şarkı sözlerini tespit etmek için oluşturulmuş bir fonksiyon.

In [6]:
def zor_kelime_bul(text):
    try:
        return textstat.difficult_words(text) # Textstat kütüphanesiyle bulunan zor kelime sayısını döndürür.
    except:
        return None # Belirlenemezse NaN döndürür.

-> Şarkı sözlerinin seviyesini belirlemek için kullanılacak formülde kullanılmak üzere zor kelime sayısını hesaplayan bir fonksiyon.

In [7]:
def seviye_puan_hesapla(text):
    try:
        return textstat.flesch_kincaid_grade(text) # Textstat kütüphanesiyle hesaplanan "Flesch Kincaid" puanını döndürür.
    except:
        return None

-> Şarkı sözlerinin seviyesini belirlemek için kullanılacak formülde kullanılmak üzere "Flesch-Kincaid" metoduyla değer hesaplayan bir fonksiyon. 

In [8]:
def seviye_hesapla(sayi):
    if sayi >= 0 and sayi <= 80:
        return "A1-A2"
    elif sayi > 80 and sayi <= 200:
        return "B1-B2"
    elif sayi > 200:
        return "C1-C2"
    else:
        return None

-> Oluşturulan formül sonucunda başka kaynaklarda kullanılan değerlerle seviye ataması yapan fonksiyon.

In [9]:
df["Binary"] = df["lyrics"].apply(lambda x: 1 if "We do not have the lyrics for" in x else 0) # Şarkı sözü olmayan verilere 1 değeri atar.
df.drop(df[df["Binary"] == 1].index, inplace=True) # Bir değerine sahip olan verileri setten siler.
df.drop("Binary", axis=1, inplace=True) # Binary sütununu siler.

-> Şarkı sözleriyle işlem yapacağımız için şarkı sözü bulunmayan verileri siliyoruz.

In [10]:
df["Temiz Metin"] = df["lyrics"].apply(metin_temizle) # Metin temizleme fonksiyonunu çalıştırır.

-> Oluşturulan "Temiz Metin" sütununa şarkı sözlerini temizlenmiş versiyonlarını ekler.

In [11]:
df["Dil"] = df["Temiz Metin"].apply(dil_bul) # Dil bulma fonksiyonunu çalıştırır.

-> Kütüphanelere uyumlu olmayan dilleri tespit etmek için temizlenmiş metinlerin dillerini bulur.

In [12]:
df["Dil"].unique() # Özgün her bir veriyi listeler.

array(['en', 'nl'], dtype=object)

-> Bulunan dilleri listeler. Kullanılan kütüphaneler listelenen dillerden sadece İngilizceye uygundur. Bu nedenle Flemenkçe şarkılar listede bulunmamalıdır.

In [13]:
df.drop(df[df["Dil"] == "nl"].index, inplace=True) # Flemenkçe verileri siler.
df["Dil"].unique()

array(['en'], dtype=object)

-> Kullanılamayacak diller silini ve kontrol edilir.

In [14]:
df["Zor Kelime"] = df["Temiz Metin"].apply(zor_kelime_bul) # Zor kelime sayısını hesaplatır.
df.head()

,genre,lyrics,SongInfo,Temiz Metin,Dil,Zor Kelime
0,Christian,"Who am I, that the Lord of all the earth Woul...",CASTING CROWNS - WHO AM I LYRICS,who am i that the lord of all the earth would...,en,5
1,Christian,Glory Revealed By His Wounds He was pierced ...,GLORY REVEALED - BY HIS WOUNDS LYRICS,glory revealed by his wounds he was pierced ...,en,5
2,Christian,Lord of heaven and earth Lord of all creation...,CAEDMON'S CALL - GOD OF WONDERS LYRICS,lord of heaven and earth lord of all creation...,en,11
3,Christian,I can only imagine what it will be like When ...,MERCYME - I CAN ONLY IMAGINE LYRICS,i can only imagine what it will be like when ...,en,8
4,Christian,I am not skilled to understand What God has w...,AARON SHUST - MY SAVIOR MY GOD LYRICS,i am not skilled to understand what god has w...,en,4


In [15]:
df["Seviye Puanı"] = df["Temiz Metin"].apply(seviye_puan_hesapla) # Flesch Kincaid methonuna göre puan hesaplatır.
df.head()

,genre,lyrics,SongInfo,Temiz Metin,Dil,Zor Kelime,Seviye Puanı
0,Christian,"Who am I, that the Lord of all the earth Woul...",CASTING CROWNS - WHO AM I LYRICS,who am i that the lord of all the earth would...,en,5,121.4
1,Christian,Glory Revealed By His Wounds He was pierced ...,GLORY REVEALED - BY HIS WOUNDS LYRICS,glory revealed by his wounds he was pierced ...,en,5,77.3
2,Christian,Lord of heaven and earth Lord of all creation...,CAEDMON'S CALL - GOD OF WONDERS LYRICS,lord of heaven and earth lord of all creation...,en,11,84.4
3,Christian,I can only imagine what it will be like When ...,MERCYME - I CAN ONLY IMAGINE LYRICS,i can only imagine what it will be like when ...,en,8,107.0
4,Christian,I am not skilled to understand What God has w...,AARON SHUST - MY SAVIOR MY GOD LYRICS,i am not skilled to understand what god has w...,en,4,140.1


In [16]:
df["Çarpım"] = df["Zor Kelime"] * df["Seviye Puanı"] / 10 # Dil düzeyi hesaplamak için farklı kaynaklardan yararlanılarak oluşturulmuş bir formüldür.
df["Çarpım"] = df["Çarpım"].astype(int) # Değerlerin tam sayı olarak işlenmesini sağlar. 
df.head()

,genre,lyrics,SongInfo,Temiz Metin,Dil,Zor Kelime,Seviye Puanı,Çarpım
0,Christian,"Who am I, that the Lord of all the earth Woul...",CASTING CROWNS - WHO AM I LYRICS,who am i that the lord of all the earth would...,en,5,121.4,60
1,Christian,Glory Revealed By His Wounds He was pierced ...,GLORY REVEALED - BY HIS WOUNDS LYRICS,glory revealed by his wounds he was pierced ...,en,5,77.3,38
2,Christian,Lord of heaven and earth Lord of all creation...,CAEDMON'S CALL - GOD OF WONDERS LYRICS,lord of heaven and earth lord of all creation...,en,11,84.4,92
3,Christian,I can only imagine what it will be like When ...,MERCYME - I CAN ONLY IMAGINE LYRICS,i can only imagine what it will be like when ...,en,8,107.0,85
4,Christian,I am not skilled to understand What God has w...,AARON SHUST - MY SAVIOR MY GOD LYRICS,i am not skilled to understand what god has w...,en,4,140.1,56


In [17]:
df["Kategori"] = df["Çarpım"].apply(seviye_hesapla) # Seviye hesaplama fonksiyonunu çalıştırır.
df.groupby("Kategori").size() # Kategoriye göre gruplanmış veri sayısını gösterir.

Kategori
A1-A2    269
B1-B2    151
C1-C2    116
dtype: int64

-> Seviyeleri hesaplar ve seviyeleri heaplanmış şarkıların sayısını görüntüler.

In [18]:
df[["Sanatçı", "Şarkı"]] = df["SongInfo"].str.split(" - ", n=1, expand=True)
df = df.drop(["lyrics", "SongInfo","Dil","Seviye Puanı","Çarpım","Zor Kelime"], axis=1)
df["Şarkı"] = df["Şarkı"].str.replace(" LYRICS", "")
df = df.reset_index(drop=True)
df.head()

,genre,Temiz Metin,Kategori,Sanatçı,Şarkı
0,Christian,who am i that the lord of all the earth would...,A1-A2,CASTING CROWNS,WHO AM I
1,Christian,glory revealed by his wounds he was pierced ...,A1-A2,GLORY REVEALED,BY HIS WOUNDS
2,Christian,lord of heaven and earth lord of all creation...,B1-B2,CAEDMON'S CALL,GOD OF WONDERS
3,Christian,i can only imagine what it will be like when ...,B1-B2,MERCYME,I CAN ONLY IMAGINE
4,Christian,i am not skilled to understand what god has w...,A1-A2,AARON SHUST,MY SAVIOR MY GOD


-> Kullanılacak veri setinin son halini görüntüler. İşlenmiş veriler tablodan çıkartılır.

-------------------------------------------------------------------------------------------------------------------------------------------------

# Kullanıcı Girişi ve Seviye Bilgisi Toplama

In [19]:
user_df = pd.read_excel("kullanıcılar.xlsx", dtype={"Şifre": str})
user_df["Kategori"] = float("NaN")
user_df

,Kullanıcı Adı,Şifre,Kategori
0,Ayşe,55652693,NaN
1,Zeynep,20431998,NaN


-> Kullanıcılar giriş verilerinin bulunduğu bir veri setidir.

In [20]:
kullanici=""

def login(): # Giriş fonksiyonu

    global username
    username = entry_username.get() # Kullanıcının girdiği kullanıcı adını alır.
    password = entry_password.get() # Kullanıcının girdiği şifreyi alır.

    # Excel dosyasını okuma
    try:
        user_df = pd.read_excel("kullanıcılar.xlsx", dtype={"Şifre": str})
        user_data = user_df.set_index("Kullanıcı Adı")[["Şifre"]].T.to_dict("list")
    except Exception as e:
        messagebox.showerror("Hata", "Excel dosyası okunurken bir hata oluştu.")
        return

    # Kullanıcı adı ve şifreyi kontrol etme
    if username in user_data:
        if user_data[username][0] == password:
            messagebox.showinfo("Başarılı", "Giriş Başarılı!")
            gizli_frame.grid_forget()  # Giriş bileşenlerini gizle
            seviye_sor(username)
            
        else:
            messagebox.showerror("Hata", "Geçersiz Şifre!")
    else:
        messagebox.showerror("Hata", "Geçersiz Kullanıcı Adı!")

-> Kullanıcı girşinin yapılması için tanımlanmış bir giriş fonksiyonudur.

In [21]:
def seviye_sor(username):
    def on_button_click(seviye):
        try:
            user_df.loc[user_df["Kullanıcı Adı"] == username, "Kategori"] = None # Uygun kategori sekmesini boşaltır.
            user_df.loc[user_df["Kullanıcı Adı"] == username, "Kategori"] = seviye # Uygun kategori sekmesine kullanıcıdan alınan seviye bilgisini yazdırır.
            user_df.to_excel("kullanıcılar.xlsx", index=False) # Kayıt eder.
            messagebox.showinfo("Başarılı", f"'{seviye}' kullanıcı dosyasına eklendi!") # Kullanıcıya mesaj kutusuyla geri bildirim verir.
            kullanici= username # Kullanıcı değişkenini girilen kullanıcı adına eşitler.
            root.destroy()
            
        except Exception as e:
            messagebox.showerror("Hata", f"Bir hata oluştu: {str(e)}")

    seviye_frame.grid(row=3, column=0, columnspan=3, padx=5, pady=5)  # Seviye seçim frame"ini göster

    button_a = tk.Button(seviye_frame, text="Başlangıç", command=lambda: on_button_click("A1-A2")) # Başlangıç seviyesi butonu
    button_a.grid(row=0, column=0, padx=5, pady=5)
    button_b = tk.Button(seviye_frame, text="Orta", command=lambda: on_button_click("B1-B2")) # Orta seviye butonu
    button_b.grid(row=0, column=1, padx=5, pady=5)
    button_c = tk.Button(seviye_frame, text="Gelişmiş", command=lambda: on_button_click("C1-C2")) # Gelişmiş seviye butonu
    button_c.grid(row=0, column=2, padx=5, pady=5)

-> Kullanıcıya sahip olduğu "İngilizce Yeterlilik Seviyesi"ni sorar ve aldığı cevabı kullanıcının dosyasına ekler.

In [22]:
# Tkinter penceresini oluşturma
root = tk.Tk()
root.title("Kullanıcı Girişi")

# Giriş bileşenlerini içeren frame
gizli_frame = tk.Frame(root)
gizli_frame.grid(row=0, column=0)

# Kullanıcı adı ve şifre girişi için etiketler
label_username = tk.Label(gizli_frame, text="Kullanıcı Adı:")
label_username.grid(row=0, column=0, padx=5, pady=5)
label_password = tk.Label(gizli_frame, text="Şifre:")
label_password.grid(row=1, column=0, padx=5, pady=5)

# Kullanıcı adı ve şifre girişi için metin kutuları
entry_username = tk.Entry(gizli_frame)
entry_username.grid(row=0, column=1, padx=5, pady=5)
entry_password = tk.Entry(gizli_frame, show="*")
entry_password.grid(row=1, column=1, padx=5, pady=5)

username = entry_username.get()
password = entry_password.get()

# Giriş butonu
button_login = tk.Button(gizli_frame, text="Giriş", command=login)
button_login.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

# Seviye seçimi bileşenlerini içeren frame
seviye_frame = tk.Frame(root)

-> Kullanıcı arayüzünün ayarlandığı kısım.

In [23]:
# Pencereyi görüntüleme
root.mainloop()

C:\Users\arifyılmaz\AppData\Local\Temp\ipykernel_12380\1203568304.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'B1-B2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  user_df.loc[user_df["Kullanıcı Adı"] == username, "Kategori"] = seviye # Uygun kategori sekmesine kullanıcıdan alınan seviye bilgisini yazdırır.


In [24]:
user_df

,Kullanıcı Adı,Şifre,Kategori
0,Ayşe,55652693,B1-B2
1,Zeynep,20431998,NaN


-> Veri girişinden sonra veri setinin son görünümüdür.

-------------------------------------------------------------------------------------------------------------------------------------------------

# Öneri Sistemi

In [25]:
gecmis = pd.read_excel("dinleme_gecmisi.xlsx", sheet_name=username) # Kullanıcıya ait dinleme geçmişini değişkene atar.
gecmis = gecmis.groupby("Şarkı")["Dinleme Süresi"].sum().reset_index() # Birden fazla kez dinlenmiş şarkıların dinlenme sürelerini toplar.
gecmis = gecmis.sort_values(by="Dinleme Süresi", ascending=False) # En çok dinlene şarkıya göre sıralar.
gecmis.head()

,Şarkı,Dinleme Süresi
96,335,1444
132,449,1057
35,109,901
154,509,901
3,10,898


-> Kullanıcya öneri sağlarken en çok dinlediği şarkının verisinden yararlanacağız.

In [26]:
ingilizce_seviyesi = user_df.loc[user_df["Kullanıcı Adı"] == username, "Kategori"].values[0] # İngilizce seviyesini değişkene atar.

top_dinlenen = gecmis.iloc[0]["Şarkı"] # En çok dinlenen şarkının index numarasını alır.

sarkı = df.loc[top_dinlenen, "Şarkı"] # İndex numarasını şarkının ismiyle eşleştirip değişkene atar.

print(sarkı)

WONDERWALL


-> Veri setinden kullanıcının en çok dinlediği şarkının ismini bulur.

In [27]:
tvf = TfidfVectorizer(min_df=1,max_df=3, max_features=350,ngram_range=(1,10),strip_accents="unicode",analyzer="word") # TfidfVectorizer sınıfı belgelerdeki terimlerin frekansını ölçerek belgeleri vektörlere dönüştürür.
df["Temiz Metin"] = df["Temiz Metin"].fillna("") # NaN değerleri boş bir karakter dizisiyle doldurur.

-> Belgeleri vektörlere dönüştürür ve olası boş değerleri belirler.

In [28]:
x=df["Temiz Metin"]

tvf_matrix_list = [] #  Boş bir liste oluşturur.
for text in x:
    tvf_matrix = tvf.fit_transform([text]) # TfidfVectorizer'ı kullanarak tek bir metin belgesini vektöre dönüştürür ve TF-IDF matrisini oluşturur.
    tvf_matrix_list.append(tvf_matrix) # Oluşturulan TF-IDF matrisini tvf_matrix_list listesine ekler.
combined_tvf_matrix = sparse.vstack(tvf_matrix_list) # Listesindeki tüm TF-IDF matrislerini birleştirerek birleşik bir TF-IDF matrisi oluşturur.
tvf_matrix = tvf.fit_transform(x) # Tüm "Temiz Metin" sütunundaki metin belgelerini bir araya getirerek tek bir TF-IDF matrisi oluşturur.
combined_tvf_matrix

<536x350 sparse matrix of type '<class 'numpy.float64'>'
	with 187600 stored elements in Compressed Sparse Row format>

-> Sigmoid metodunda kullanılacak matrixleri hazırlar.

In [29]:
sig = sigmoid_kernel(combined_tvf_matrix, combined_tvf_matrix)

-> Sigmoid iki belge arasındaki benzerliği ölçmek için kullanılan bir kernel fonksiyonudur.

In [30]:
indicades = pd.Series(df.index , index= df["Şarkı"]).drop_duplicates()

In [31]:
indicades

Şarkı
WHO AM I                    0
BY HIS WOUNDS               1
GOD OF WONDERS              2
I CAN ONLY IMAGINE          3
MY SAVIOR MY GOD            4
                         ... 
COLD SWEAT                531
RAINY NIGHT IN GEORGIA    532
TAKE ME TO THE RIVER      533
C.C. RIDER                534
DANCE TO THE MUSIC        535
Length: 536, dtype: int64

-> Şarkı isimlerini ait oldukları index numarasını indicades değişkenine atar.

In [32]:
def oneri_yap (title , sig=sig):
    idx = indicades.loc[title] # Verilen şarkı için indeksi belirler.

    sig_skor = list(enumerate(sig[idx])) # İndeks kullanarak, verilen şarkı için benzerlik skorlarını  alır ve bunları numaralandırır.

    sig_skor = sorted(sig_skor, key=lambda x: x[1], reverse=True) # Benzerlik skorlarını, skorlarına göre büyükten küçüğe sıralar.

    sig_skor = sig_skor[1:536] # En yüksek olanı(kendisi) hariç tüm skorları seçer.

    song_indicades = [i[0] for i in sig_skor] # Önerilen şarkıların indekslerini alır.

    return df.iloc[song_indicades][["Şarkı", "Sanatçı", "genre", "Kategori"]] # Önerilen şarkıların bilgilerini içeren bir DataFrame döndürür.

-> Sigmoid metoduyla hesaplana benzerlik değerlerine göre öneri listesi oluşturan fonksiyon.

In [33]:
oneri_listesi = oneri_yap(sarkı)
oneri_listesi

,Şarkı,Sanatçı,genre,Kategori
102,IT WASN'T GOD WHO MADE HONKY TONK ANGELS,KITTY WELLS,Country,A1-A2
11,WORD OF GOD SPEAK,MERCYME,Christian,A1-A2
115,HE'LL HAVE TO GO,JIM REEVES,Country,A1-A2
396,MADHOUSE,ANTHRAX,Rock,A1-A2
129,WALK ON BY,LEROY VAN DYKE,Country,A1-A2
...,...,...,...,...
364,I WANNA ROCK,TWISTED SISTER,Rock,A1-A2
510,RAPPER'S DELIGHT,THE SUGAR HILL GANG,R&B,C1-C2
366,BREAKING THE LAW,JUDAS PRIEST,Rock,A1-A2
254,WHO'S THAT GIRL?,EVE,Hip-Hop,C1-C2


-> Oluşturulmuş öneri listesi.

In [34]:
oneri_listesi = oneri_listesi.loc[oneri_listesi["Kategori"] == ingilizce_seviyesi] # Öneri listesini kullanıcının seviyesine göre daraltır.
kullaniciya_sunulacak = oneri_listesi.head(20) # Kullanıcıya sunulacak verileri toplar.

-> Öneri listesini kullanıcının seviyesine göre sınıflandırır ve seçer.

In [35]:
root = tk.Tk()
root.title("Öneriler") # Sekmenin başlığını oluşturur.

table = ttk.Treeview(root) # Tablo oluşturur.
table["columns"] = list(kullaniciya_sunulacak) 
table["show"] = "headings"

for column in kullaniciya_sunulacak: # Veri başlıklarını tabloya yerleştirir.
    table.heading(column, text=column)

for index, row in kullaniciya_sunulacak.iterrows(): # Verileri tabloya yerleştirir.
    table.insert("", "end", values=list(row))

table.pack(expand=True, fill="both")

-> Kullanıcıya önerilen şarkıları bir ayarüz yardımıyla gösterir.

In [36]:
root.mainloop()